In [1]:
# Import Libraries

import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import pandas as pd
import matplotlib as mpl
# import chart_studio.plotly as py
import numpy as np
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# from geopy import Nominatim
# from geopy.extra.rate_limiter import RateLimiter
%matplotlib inline

In [2]:
# Run the file Data_Preprocessing.ipynb which cleaned the data. Pre-processed data are saved in the data file

housing = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col = 0)
real_estate = pd.read_csv('data/Ames_Real_Estate_Data.csv', index_col = 0)
# combined data set:
ames_df = pd.read_csv('data/final_df.csv', index_col = 0)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
categorical = ['MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 
               'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
               'Electrical', 'LowQualFinSF', 'KitchenQual', 'Functional']

for feature in categorical:
    print(ames_df[feature].value_counts())
    print('')

None       1428
BrkFace     769
Stone       180
BrkCmn       17
Name: MasVnrType, dtype: int64

TA    1484
Gd     826
Ex      73
Fa      24
Name: ExterQual, dtype: int64

TA    2091
Gd     256
Fa      49
Ex       9
Po       2
Name: ExterCond, dtype: int64

PConc     1070
CBlock    1057
BrkTil     229
Slab        38
Stone        9
Wood         4
Name: Foundation, dtype: int64

TA    1064
Gd    1031
Ex     187
Fa      69
Po       2
Name: BsmtQual, dtype: int64

TA    2180
Gd      94
Fa      74
Ex       3
Po       2
Name: BsmtCond, dtype: int64

No    1586
Av     329
Gd     234
Mn     202
Name: BsmtExposure, dtype: int64

GLQ    727
Unf    646
ALQ    381
Rec    244
BLQ    235
LwQ    120
Name: BsmtFinType1, dtype: int64

Unf    2050
Rec      89
LwQ      79
BLQ      57
ALQ      52
GLQ      25
Name: BsmtFinType2, dtype: int64

GasA     2374
GasW       22
Grav        4
Wall        4
OthW        2
Floor       1
Name: Heating, dtype: int64

Ex    1234
TA     701
Gd     404
Fa      67
Po       1

### Q3. Preparing the data for Multiple Linear Regression
- Decide which of the features is quantitative, which are nominal categorical, which are ordinal categorical
- We plan to run a linear regression of 'SalesPrice' against features in 
        ['MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional']
- To do so, we need to 
    - convert string binary categorical values into 0 and 1
    - dummify nominal categorical features (droping the dominant category)

Excluding **ID**, there are 14 distinct features, they can be classified into quantitative, ordinal and nominal categorical types

- Quantitative
   - BirthWeightOz & BirthWeightGm (they can also be viewed as ordinal categorical)
   - MomAge (can be treated as ordinal categorical) 
   - Gained
    
- Ordinal Categorical
   - Weeks, Plural
    
- Nominal Categorical
   - Sex, Marital, HispMom, Smoke, Low, Premie, MomRace

In [11]:
ames_df2 = ames_df[['SalePrice', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
 '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
 'TotRmsAbvGrd', 'Functional']]

In [6]:
ames_df2.head()

,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,...,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional
0,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,...,0,1.0,0.0,1,0,2,1,TA,4,Typ
1,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,...,0,1.0,0.0,2,0,2,1,Gd,5,Typ
2,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,...,0,1.0,0.0,2,0,2,1,Gd,5,Typ
3,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,...,0,0.0,0.0,1,0,2,1,Gd,5,Typ
4,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,...,0,0.0,0.0,1,0,2,1,TA,6,Typ


In [64]:
features      = ames_df2[['MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
 '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
 'TotRmsAbvGrd', 'Functional']].copy()
sale_price    = ames_df2['SalePrice']

### Dummy Variables

In [65]:
# MasVnrType
dummy         = pd.get_dummies(features.MasVnrType, prefix='MasVnrType')
dummy.drop('MasVnrType_BrkCmn', axis=1, inplace=True) 
features = features.drop('MasVnrType', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,...,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone
0,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,...,0.0,1,0,2,1,4,Typ,0,1,0
1,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,...,0.0,2,0,2,1,5,Typ,1,0,0
2,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,...,0.0,2,0,2,1,5,Typ,1,0,0
3,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,...,0.0,1,0,2,1,5,Typ,0,1,0
4,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,...,0.0,1,0,2,1,6,Typ,0,1,0


In [66]:
# ExterQual
dummy         = pd.get_dummies(features.ExterQual, prefix='ExterQual')
dummy.drop('ExterQual_Ex', axis=1, inplace=True) 
features = features.drop('ExterQual', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA
0,0.0,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,...,2,1,4,Typ,0,1,0,0,0,1
1,149.0,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,...,2,1,5,Typ,1,0,0,0,1,0
2,149.0,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,...,2,1,5,Typ,1,0,0,0,1,0
3,0.0,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,...,2,1,5,Typ,0,1,0,0,1,0
4,0.0,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,...,2,1,6,Typ,0,1,0,0,1,0


In [67]:
# ExterCond
dummy         = pd.get_dummies(features.ExterCond, prefix='ExterCond')
dummy.drop('ExterCond_Ex', axis=1, inplace=True) 
features = features.drop('ExterCond', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,...,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA
0,0.0,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,...,0,1,0,0,0,1,0,0,0,1
1,149.0,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,...,1,0,0,0,1,0,0,0,0,1
2,149.0,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,...,1,0,0,0,1,0,0,0,0,1
3,0.0,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,...,0,1,0,0,1,0,0,0,0,1
4,0.0,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,...,0,1,0,0,1,0,0,1,0,0


In [68]:
# Foundation
dummy         = pd.get_dummies(features.Foundation, prefix='Foundation')
dummy.drop('Foundation_BrkTil', axis=1, inplace=True)
features = features.drop('Foundation', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
0,0.0,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,...,1,0,0,0,1,1,0,0,0,0
1,149.0,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,...,0,0,0,0,1,1,0,0,0,0
2,149.0,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,...,0,0,0,0,1,1,0,0,0,0
3,0.0,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,...,0,0,1,0,0,0,0,0,0,0


In [69]:
# BsmtQual
dummy         = pd.get_dummies(features.BsmtQual, prefix='BsmtQual')
dummy.drop('BsmtQual_Ex', axis=1, inplace=True) 
features = features.drop('BsmtQual', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Po,BsmtQual_TA
0,0.0,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,...,1,1,0,0,0,0,0,0,0,1
1,149.0,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,...,1,1,0,0,0,0,0,1,0,0
2,149.0,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,...,1,1,0,0,0,0,0,1,0,0
3,0.0,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,...,1,0,0,0,0,0,0,0,0,1
4,0.0,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,...,0,0,0,0,0,0,1,0,0,0


In [70]:
# BsmtCond
dummy         = pd.get_dummies(features.BsmtCond, prefix='BsmtCond')
dummy.drop('BsmtCond_Ex', axis=1, inplace=True) 
features = features.drop('BsmtCond', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Po,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA
0,0.0,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,...,0,0,0,0,0,1,0,0,0,1
1,149.0,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,...,0,0,0,1,0,0,0,0,0,1
2,149.0,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,...,0,0,0,1,0,0,0,0,0,1
3,0.0,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,...,0,0,0,0,0,1,0,0,0,1
4,0.0,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,...,0,0,1,0,0,0,0,0,0,1


In [71]:
# BsmtExposure
dummy         = pd.get_dummies(features.BsmtExposure, prefix='BsmtExposure')
dummy.drop('BsmtExposure_Av', axis=1, inplace=True) 
features = features.drop('BsmtExposure', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,...,BsmtQual_Gd,BsmtQual_Po,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,...,0,0,1,0,0,0,1,0,0,1
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,...,1,0,0,0,0,0,1,0,1,0
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,...,1,0,0,0,0,0,1,0,1,0
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,...,0,0,1,0,0,0,1,0,0,1
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,...,0,0,0,0,0,0,1,0,0,1


In [72]:
# Heating
dummy         = pd.get_dummies(features.Heating, prefix='Heating')
dummy.drop('Heating_Floor', axis=1, inplace=True) 
features = features.drop('Heating', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,...,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,TA,Y,SBrkr,...,0,1,0,0,1,1,0,0,0,0
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,TA,Y,SBrkr,...,0,1,0,1,0,1,0,0,0,0
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,TA,Y,SBrkr,...,0,1,0,1,0,1,0,0,0,0
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,Ex,Y,SBrkr,...,0,1,0,0,1,1,0,0,0,0
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,Gd,Y,SBrkr,...,0,1,0,0,1,1,0,0,0,0


In [73]:
# HeatingQC
dummy         = pd.get_dummies(features.HeatingQC, prefix='HeatingQC')
dummy.drop('HeatingQC_Ex', axis=1, inplace=True) 
features = features.drop('HeatingQC', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,Electrical,1stFlrSF,...,BsmtExposure_No,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,Y,SBrkr,856,...,1,1,0,0,0,0,0,0,0,1
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,Y,SBrkr,1049,...,0,1,0,0,0,0,0,0,0,1
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,Y,SBrkr,1049,...,0,1,0,0,0,0,0,0,0,1
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,Y,SBrkr,1001,...,1,1,0,0,0,0,0,0,0,0
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,Y,SBrkr,717,...,1,1,0,0,0,0,0,1,0,0


In [74]:
# CentralAir
dummy         = pd.get_dummies(features.CentralAir, prefix='CentralAir')
dummy.drop('CentralAir_N', axis=1, inplace=True) 
features = features.drop('CentralAir', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Electrical,1stFlrSF,2ndFlrSF,...,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,SBrkr,856,0,...,1,0,0,0,0,0,0,0,1,1
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,SBrkr,1049,0,...,1,0,0,0,0,0,0,0,1,1
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,SBrkr,1049,0,...,1,0,0,0,0,0,0,0,1,1
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,SBrkr,1001,0,...,1,0,0,0,0,0,0,0,0,1
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,SBrkr,717,322,...,1,0,0,0,0,0,1,0,0,1


In [75]:
# Electrical
dummy         = pd.get_dummies(features.Electrical, prefix='Electrical')
dummy.drop('Electrical_FuseA', axis=1, inplace=True) 
features = features.drop('Electrical', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,856,0,0,...,0,0,0,0,0,1,1,0,0,1
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,1049,0,0,...,0,0,0,0,0,1,1,0,0,1
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,1049,0,0,...,0,0,0,0,0,1,1,0,0,1
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,1001,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,717,322,0,...,0,0,0,1,0,0,1,0,0,1


In [76]:
# Functional
dummy         = pd.get_dummies(features.Functional, prefix='Functional')
dummy.drop('Functional_Maj1', axis=1, inplace=True) 
features = features.drop('Functional', axis=1)
features = pd.concat([features, dummy], axis=1)
features.head()

,MasVnrArea,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ
0,0.0,Rec,238.0,Unf,0.0,618.0,856.0,856,0,0,...,1,0,0,1,0,0,0,0,0,1
1,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,1049,0,0,...,1,0,0,1,0,0,0,0,0,1
2,149.0,GLQ,552.0,ALQ,393.0,104.0,1049.0,1049,0,0,...,1,0,0,1,0,0,0,0,0,1
3,0.0,ALQ,737.0,Unf,0.0,100.0,837.0,1001,0,0,...,1,0,0,1,0,0,0,0,0,1
4,0.0,Unf,0.0,Unf,0.0,405.0,405.0,717,322,0,...,1,0,0,1,0,0,0,0,0,1


### Machine Learning with Multiple Linear Regression 
- Now run a multiple linear regression model of birth weights (in grams) 
against the birth features created in the previous step
- Report the $R^2$ score on the train set
- Investigate the histogram of the residuals using plt.hist function
- Compute the **RSS** using numpy and the model's predict method
- Interpret the meaning of the **MomAge** slope coefficient
- Report the intercept and slope coefficients of the linear model
- Report the column names with negative and positive slope coefficients

In [78]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(features, sale_price)
residuals = sale_price - lm.predict(features)
plt.hist(residuals)

ValueError: could not convert string to float: 'Rec'

In [87]:
features      = ames_df[['MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
 'BsmtFinType2', 'BasmtFinSF', 'BsmtUnfSF', 'TotalBsmtSF',
 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
 '2ndFlrSF', 'LowQualFinSF', 'FullBath',
 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'num_bathroom',
 'TotRmsAbvGrd', 'Functional']].copy()
sale_price    = ames_df2['SalePrice']

In [88]:
housing_featuresClass = features[['MasVnrType', 'ExterQual', 'ExterCond',
 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'Functional']]
housing_featuresQuan = features[['MasVnrArea', 'BasmtFinSF', 'BsmtUnfSF', 'TotalBsmtSF',
 '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'num_bathroom', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd']]

In [89]:
features.isnull().sum()

MasVnrType      13
MasVnrArea      13
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual        54
BsmtCond        54
BsmtExposure    56
BsmtFinType1    54
BsmtFinType2    55
BasmtFinSF       0
BsmtUnfSF        1
TotalBsmtSF      1
Heating          0
HeatingQC        0
CentralAir       0
Electrical       1
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
num_bathroom     2
TotRmsAbvGrd     0
Functional       0
dtype: int64

In [ ]:
# Drop the Nan from GarageCars column (1 observation)
features.drop(features[features['GarageCars'].isnull() == True].index,inplace=True)
features.columns